In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import re

In [ ]:
def all_stock_metadata(div_tag):
  data = []
  for item in div_tag.find_all('li'):
      text_data = item.text.strip()
      data.append(text_data)
  keys = []
  values = []

  for item in data:
      parts = item.split('\n')

      key = parts[0].strip()
      value = ' '.join(parts[2:]).strip()

      formatted_key = key
      formatted_value = ' '.join(value.split())

      keys.append(formatted_key)
      values.append(formatted_value)

  df = pd.DataFrame(values , keys)
  df1 = df.T
  return df1

In [ ]:
url = 'https://www.screener.in/company/VOLTAS'
response = requests.get(url)
content = response.content
all_data = pd.read_html(content)


[           Unnamed: 0 Mar 2022 Jun 2022 Sep 2022 Dec 2022 Mar 2023 Jun 2023  \
 0             Sales +     2522     2592     1409     1423     2252     2699   
 1          Expenses +     2259     2411     1335     1383     2119     2522   
 2    Operating Profit      262      180       74       40      133      177   
 3               OPM %      10%       7%       5%       3%       6%       7%   
 4      Other Income +       38       27     1009       33       81      106   
 5            Interest        9        1        2        2        7        3   
 6        Depreciation        8        8        9       10        9       10   
 7   Profit before tax      283      198     1072       61      198      270   
 8               Tax %      23%      26%       1%      27%      21%      20%   
 9        Net Profit +      216      147     1058       44      157      215   
 10          EPS in Rs     6.54     4.43    31.97     1.34     4.73     6.49   
 11            Raw PDF      NaN      NaN

In [ ]:
def initial(x):
  response = requests.get(x)
  content = response.content
  all_data = pd.read_html(content)
  return all_data

In [ ]:
def process_holdings(all_data):
    all_data[9].set_index('Unnamed: 0' , inplace = True)
    holdings = all_data[9].iloc[:5, -1]
    df2 = pd.DataFrame(holdings)
    df2 = df2.T
    df2.reset_index(drop=True, inplace=True)
    return df2
process_holdings(all_data)

Unnamed: 0,Promoters +,FIIs +,DIIs +,Government +,Public +
0,30.30%,21.95%,32.99%,0.21%,14.53%


In [ ]:
holdings = (all_data[9].iloc[:5, -1])
df2 = pd.DataFrame(holdings)
df2 = df2.T
df2.reset_index(drop=True, inplace=True)
df2

Unnamed: 0,Promoters +,FIIs +,DIIs +,Government +,Public +
0,30.30%,21.95%,32.99%,0.21%,14.53%


In [ ]:
all_data[0].set_index("Unnamed: 0", inplace=True)
df3 = all_data[0].iloc[:, -1]
df3 = pd.DataFrame(df3)
df3 = df3.T
df3 = df3.reset_index(drop = True)
df3

Unnamed: 0,Sales +,Expenses +,Operating Profit,OPM %,Other Income +,Interest,Depreciation,Profit before tax,Tax %,Net Profit +,EPS in Rs,Raw PDF
0,3635,3393,242,7%,89,17,13,301,27%,220,6.66,NaN


In [ ]:
def process_dataframe(data):
    data.set_index("Unnamed: 0", inplace=True)
    df3 = data.iloc[:, -1]
    df3 = pd.DataFrame(df3).T
    df3 = df3.reset_index(drop=True)
    return df3

In [ ]:
def get_stock_data(screener_url):
  response = requests.get(screener_url)
  soup = BeautifulSoup(response.text, 'html.parser')
  content = response.content
  all_data = pd.read_html(content)
  div_tag =  soup.find(id="top-ratios")
  df1 = all_stock_metadata(div_tag)
  df2 = process_holdings(all_data)
  df3 = process_dataframe(all_data[0])
  return pd.concat([df1, df2 ,df3], axis=1)
get_stock_data('https://www.screener.in/company/VOLTAS/consolidated/')


,Market Cap,Current Price,High / Low,Stock P/E,Book Value,Dividend Yield,ROCE,ROE,Face Value,Promoters +,...,Operating Profit,OPM %,Other Income +,Interest,Depreciation,Profit before tax,Tax %,Net Profit +,EPS in Rs,Raw PDF
0,"₹ 42,697 Cr.","₹ 1,290","₹ 1,946 / 1,135",51.2,₹ 197,0.43 %,17.9 %,13.5 %,₹ 1.00,30.30%,...,301,6%,80,23,14,343,31%,236,7.28,NaN


In [ ]:
failed = []
all_stock_tickers = pd.read_excel('/content/all_stocks.xlsx')
all_stock_tickers_list = all_stock_tickers['Symbol'].values.tolist()

all_stocks_data_df = pd.DataFrame()

for ticker in tqdm(all_stock_tickers_list):
  try:
    stock_data_df = get_stock_data(f'https://www.screener.in/company/{ticker}/')
    stock_data_df['Ticker'] = ticker
    all_stocks_data_df = pd.concat([all_stocks_data_df, stock_data_df])
  except:
    print(ticker)
    failed.append(ticker)
    continue

  2%|▏         | 45/2381 [00:48<37:13,  1.05it/s]

ZOMATO


  4%|▍         | 104/2381 [01:53<39:40,  1.05s/it]

MCDOWELL-N


  7%|▋         | 158/2381 [02:52<36:14,  1.02it/s]

...

VIVIMEDLAB


100%|█████████▉| 2380/2381 [39:53<00:00,  1.12it/s]

nan


100%|██████████| 2381/2381 [39:54<00:00,  1.01s/it]

Note: Kindly refer regulation 15 to 27 of SEBI (Listing Obligation and Disclosure Requirments) Regulations, 2015 for determining the applicability of other provisions of Corporate Governance.

* The list includes companies which are either suspended due to Capital Reduction/ALF/GSM/ASM IBC/non-compliance etc.


In [ ]:
all_stocks_data_df.to_excel('newstock_data.xlsx', index=False)

In [ ]:
all_stocks_data_df

,Market Cap,Current Price,High / Low,Stock P/E,Book Value,Dividend Yield,ROCE,ROE,Face Value,Promoters +,...,EPS in Rs,Raw PDF,Ticker,Revenue +,Financing Profit,Financing Margin %,Gross NPA %,Net NPA %,No. of Shareholders,Others +
0,"₹ 19,51,043 Cr.","₹ 1,442","₹ 1,609 / 1,115",55.3,₹ 401,0.35 %,7.62 %,6.66 %,₹ 10.0,50.10%,...,8.29,NaN,RELIANCE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"₹ 12,42,434 Cr.","₹ 3,434","₹ 4,592 / 3,056",25.9,₹ 209,1.75 %,78.4 %,65.0 %,₹ 1.00,71.77%,...,30.72,NaN,TCS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"₹ 14,89,422 Cr.","₹ 1,943","₹ 1,996 / 1,574",22.1,₹ 650,1.13 %,7.35 %,14.4 %,₹ 1.00,0.00%,...,23.02,NaN,HDFCBANK,77460,11316,15%,1.33%,0.43%,NaN,NaN
0,"₹ 10,16,804 Cr.","₹ 1,425","₹ 1,472 / 1,099",21.5,₹ 407,0.70 %,7.62 %,17.9 %,₹ 2.00,NaN,...,17.73,NaN,ICICIBANK,42431,9513,22%,1.67%,0.39%,1902503,NaN
0,"₹ 11,06,318 Cr.","₹ 1,844","₹ 1,917 / 1,373",55.3,₹ 246,0.43 %,10.4 %,16.6 %,₹ 5.00,52.42%,...,16.34,NaN,BHARTIARTL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,₹ 43.3 Cr.,₹ 19.2,₹ /,,₹ 327,0.00 %,-0.25 %,-7.14 %,₹ 10.0,18.09%,...,-0.26,NaN,VALECHAENG,NaN,NaN,NaN,NaN,NaN,14129,NaN
0,₹ 8.90 Cr.,₹ 2.27,₹ /,,₹ -429,0.00 %,-4.83 %,%,₹ 10.0,46.76%,...,-17.05,NaN,VALUEIND,NaN,NaN,NaN,NaN,NaN,64098,NaN
0,₹ 346 Cr.,₹ 29.7,₹ 47.2 / 19.2,,₹ -118,0.00 %,-5.46 %,%,₹ 10.0,52.66%,...,-42.16,NaN,VISASTEEL,NaN,NaN,NaN,NaN,NaN,17750,NaN
0,₹ 0.19 Cr.,₹ 0.05,₹ /,,₹ -5.19,0.00 %,%,%,₹ 10.0,14.42%,...,-0.11,NaN,VISUINTL,NaN,NaN,NaN,NaN,NaN,21555,NaN


In [ ]:
stock_data = pd.read_excel("/content/newstock_data.xlsx")

In [ ]:
stock_data.columns

Index(['Market Cap', 'Current Price', 'High / Low', 'Stock P/E', 'Book Value',
       'Dividend Yield', 'ROCE', 'ROE', 'Face Value', 'Promoters +', 'FIIs +',
       'DIIs +', 'Government +', 'Public +', 'Sales +', 'Expenses +',
       'Operating Profit', 'OPM %', 'Other Income +', 'Interest',
       'Depreciation', 'Profit before tax', 'Tax %', 'Net Profit +',
       'EPS in Rs', 'Raw PDF', 'Ticker', 'Revenue +', 'Financing Profit',
       'Financing Margin %', 'Gross NPA %', 'Net NPA %', 'No. of Shareholders',
       'Others +'],
      dtype='object')

In [ ]:
columns_to_drop = ['High / Low','Promoters\xa0+','FIIs\xa0+','DIIs\xa0+',
                   'Government\xa0+','Expenses\xa0+','Other Income\xa0+','OPM %',
                   'Financing Profit','Financing Margin %','Raw PDF', 'Gross NPA %', 'Net NPA %',
                   'No. of Shareholders','Others\xa0+']

In [ ]:
stock_data_final = stock_data.drop(columns_to_drop, axis=1)

In [ ]:
stock_data_final= stock_data_final.dropna(subset = ['Stock P/E'],axis=0)

In [ ]:
stock_data_final.to_excel('new_stock_data.xlsx', index=False)